Quick and dirty explore alternatives to screen scraping
for prefixping service. 

basicly instead of scraping the search results page  
find the yaml file used to generate the page 

for the GO registery the file is: 

[db-xrefs.yaml](http://current.geneontology.org/metadata/db-xrefs.yaml)

for the n3t/identifiers registery the file is:  

[cdl_ebi_prefixes.yaml](https://n2t.net/e/cdl_ebi_prefixes.yaml)


In [20]:
import time
import re
import requests
import yaml
import os.path
from datetime import datetime

In [2]:
gocurl = 'http://current.geneontology.org/metadata/db-xrefs.yaml'
gocfile = 'db-xrefs.yaml'
gocprefixfile = 'gocprefix.yaml'

then = time.time()
response = requests.head(gocurl)
time.time() - then

then = time.time()
response = requests.get(gocurl)
time.time() - then

`head` is faster than `get` for finding `last_modified`   
but not enough to worry about right now

In [11]:
gocprefix = []
response = requests.get(gocurl)
if response.status_code == requests.codes.ok:
    gocxref = yaml.load(response.text)
    remote_last_modified = response.headers['Last-Modified'] 
    # Tue, 01 Aug 2017 14:14:26 GMT   note see http://strftime.org/
    fmt = '%a, %d %b %Y %H:%M:%S GMT'
    remote_datetime = datetime.strptime(remote_last_modified, fmt)
    gocprefix.append('# ' + str(remote_datetime))
    for db in gocxref:
         gocprefix.append(db['database'])
else: 
    print('ERROR ' + response.url + ' returned '+ str(response.status_code))
    print('Trying local cache')
    remote_datetime = None
    if os.path.isfile(gocprefixfile):
        with open(gocprefixfile, 'r') as fh:
            gocprefix = yaml.load(fh)
        local_datetime = gocprefix[1]
        print('Found local cache from: ',local_datestamp)
    else:
        print('Error no local cache of '+ gocprefixfile + ' available')
        local_datetime = None

In [13]:
len(gocprefix)

313

In [15]:
"BUNNYFOOFOO" in gocprefix

False

In [16]:
"GO" in gocprefix

True

In [22]:
for p in gocprefix:
    if re.match(r'.*/.*', p):
        print(p)

shows these GOC prefixes are __not__ composed  
of namespaces poroceded 
by an optional " provider code"  then slash

___

## CDL/EBI

In [ ]:
https://n2t.net/e/cdl_ebi_prefixes.yaml